# Time-Adjusted Wing Model Test Suite

This notebook contains test cases for the Time-Adjusted Wing Model volatility fitter to ensure functionality and detect breaking changes.

## Test Categories
1. **Basic Functionality Tests** - Core model and parameter operations
2. **Calibration Tests** - Optimization and fitting functionality
3. **Integration Tests** - Real-world scenarios
4. **Regression Tests** - Consistency checks
5. **Performance Tests** - Speed benchmarks

In [1]:
# Setup and imports
import sys
import os
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
sys.path.append(project_root)

from utils.volatility_fitter.time_adjusted_wing_model.time_adjusted_wing_model import (
    TimeAdjustedWingModel, 
    TimeAdjustedWingModelParameters
)
from utils.volatility_fitter.time_adjusted_wing_model.time_adjusted_wing_model_calibrator import (
    TimeAdjustedWingModelCalibrator,
    TimeAdjustedCalibrationResult,
    create_time_adjusted_wing_model_from_result
)

print("✅ All modules imported successfully")
print(f"📁 Project root: {project_root}")

✅ All modules imported successfully
📁 Project root: /home/user/Python/Baseoffset-Fitting-Manager


## 1. Basic Functionality Tests

In [2]:
# Test 1: Parameter initialization and methods
print("🧪 Test 1: Parameter Initialization")

params = TimeAdjustedWingModelParameters(
    atm_vol=0.6,
    slope=0.08,
    call_curve=5.0,
    put_curve=5.0,
    up_cutoff=0.5,
    down_cutoff=-0.95,
    up_smoothing=5.0,
    down_smoothing=5.0,
    forward_price=60000.0,
    time_to_expiry=0.25
)

# Test parameter access
assert params.atm_vol == 0.6
assert params.forward_price == 60000.0
assert params.time_to_expiry == 0.25

# Test parameter methods
param_names = params.get_parameter_names()
param_values = params.get_fitted_vol_parameter()

assert len(param_names) == 8
assert len(param_values) == 8
assert 'forward_price' not in param_names
assert 'time_to_expiry' not in param_names

print(f"✅ Parameter names: {param_names}")
print(f"✅ Parameter values: {param_values}")
print("✅ Test 1 passed!")

🧪 Test 1: Parameter Initialization
✅ Parameter names: ['atm_vol', 'slope', 'call_curve', 'put_curve', 'up_cutoff', 'down_cutoff', 'up_smoothing', 'down_smoothing']
✅ Parameter values: [0.6, 0.08, 5.0, 5.0, 0.5, -0.95, 5.0, 5.0]
✅ Test 1 passed!


In [3]:
# Test 2: Model initialization and basic methods
print("🧪 Test 2: Model Initialization")

model = TimeAdjustedWingModel(params, use_norm_term=True)

# Test model attributes
assert model.parameters == params
assert model.use_norm_term == True

# Test normalization term calculation
norm_term = model.get_normalization_term(0.25)
assert isinstance(norm_term, float)
assert norm_term > 0

print(f"✅ Model initialized with norm_term={model.use_norm_term}")
print(f"✅ Normalization term: {norm_term:.4f}")
print("✅ Test 2 passed!")

🧪 Test 2: Model Initialization
✅ Model initialized with norm_term=True
✅ Normalization term: 0.1146
✅ Test 2 passed!


In [4]:
# Test 3: Volatility calculation
print("🧪 Test 3: Volatility Calculation")

# Test ATM volatility
atm_vol = model.calculate_volatility_from_strike(60000.0)
assert isinstance(atm_vol, float)
assert 0.01 < atm_vol < 2.0

# Test OTM volatilities
otm_call_vol = model.calculate_volatility_from_strike(70000.0)
otm_put_vol = model.calculate_volatility_from_strike(50000.0)

assert isinstance(otm_call_vol, float)
assert isinstance(otm_put_vol, float)
assert 0.01 < otm_call_vol < 2.0
assert 0.01 < otm_put_vol < 2.0

print(f"✅ ATM vol (60K): {atm_vol:.3f}")
print(f"✅ OTM Call vol (70K): {otm_call_vol:.3f}")
print(f"✅ OTM Put vol (50K): {otm_put_vol:.3f}")
print("✅ Test 3 passed!")

🧪 Test 3: Volatility Calculation
✅ ATM vol (60K): 0.600
✅ OTM Call vol (70K): 0.612
✅ OTM Put vol (50K): 0.631
✅ Test 3 passed!


In [5]:
# Test 4: Volatility smile shape
print("🧪 Test 4: Volatility Smile Shape")

strikes = [45000, 50000, 55000, 60000, 65000, 70000, 75000]
vols = [model.calculate_volatility_from_strike(strike) for strike in strikes]

# All volatilities should be positive and reasonable
assert all(0.01 < vol < 2.0 for vol in vols)

# Display the smile
print("Volatility Smile:")
for strike, vol in zip(strikes, vols):
    print(f"  {strike:,}: {vol:.3f} ({vol*100:.1f}%)")

# Find minimum (should be near ATM)
min_vol_idx = np.argmin(vols)
min_vol_strike = strikes[min_vol_idx]
forward_distance = abs(min_vol_strike - params.forward_price) / params.forward_price

print(f"✅ Minimum vol at strike: {min_vol_strike:,}")
print(f"✅ Distance from forward: {forward_distance:.1%}")
print("✅ Test 4 passed!")

🧪 Test 4: Volatility Smile Shape
Volatility Smile:
  45,000: 0.671 (67.1%)
  50,000: 0.631 (63.1%)
  55,000: 0.609 (60.9%)
  60,000: 0.600 (60.0%)
  65,000: 0.602 (60.2%)
  70,000: 0.612 (61.2%)
  75,000: 0.629 (62.9%)
✅ Minimum vol at strike: 60,000
✅ Distance from forward: 0.0%
✅ Test 4 passed!


In [6]:
# Test 5: Moneyness calculation
print("🧪 Test 5: Moneyness Calculation")

# ATM moneyness should be close to 0
atm_moneyness = model.calculate_moneyness(60000.0, 60000.0, 0.25, 0.6)
assert abs(atm_moneyness) < 0.2

# OTM call should be positive
otm_call_moneyness = model.calculate_moneyness(60000.0, 70000.0, 0.25, 0.6)
assert otm_call_moneyness > 0

# OTM put should be negative
otm_put_moneyness = model.calculate_moneyness(60000.0, 50000.0, 0.25, 0.6)
assert otm_put_moneyness < 0

print(f"✅ ATM moneyness: {atm_moneyness:.4f}")
print(f"✅ OTM Call moneyness: {otm_call_moneyness:.4f}")
print(f"✅ OTM Put moneyness: {otm_put_moneyness:.4f}")
print("✅ Test 5 passed!")

🧪 Test 5: Moneyness Calculation
✅ ATM moneyness: -0.0172
✅ OTM Call moneyness: 0.0417
✅ OTM Put moneyness: -0.0869
✅ Test 5 passed!


## 2. Calibration Tests

In [7]:
# Test 6: Calibrator initialization
print("🧪 Test 6: Calibrator Initialization")

calibrator = TimeAdjustedWingModelCalibrator(use_norm_term=True)

# Test attributes
assert calibrator.enable_bounds == True
assert calibrator.method == "SLSQP"
assert calibrator.use_norm_term == True

# Test parameter bounds
bounds = calibrator._get_parameter_bounds()
assert len(bounds) == 8

print(f"✅ Calibrator method: {calibrator.method}")
print(f"✅ Parameter bounds: {len(bounds)} bounds")
print("✅ Test 6 passed!")

🧪 Test 6: Calibrator Initialization
✅ Calibrator method: SLSQP
✅ Parameter bounds: 8 bounds
✅ Test 6 passed!


In [8]:
# Test 7: Loss function calculation
print("🧪 Test 7: Loss Function")

# Sample market data
strikes = [50000, 55000, 60000, 65000, 70000]
market_vols = [0.8, 0.7, 0.6, 0.7, 0.8]
vegas = [100, 150, 200, 150, 100]
weights = [1.0] * len(strikes)
forward_price = 60000.0
time_to_expiry = 0.25

# Test loss function
params_test = [0.6, 0.08, 5.0, 5.0, 0.5, -0.95, 5.0, 5.0]
args = (strikes, market_vols, vegas, weights, forward_price, time_to_expiry, True)

loss = calibrator._loss_function(params_test, *args)

assert isinstance(loss, float)
assert loss >= 0
assert not np.isnan(loss)
assert not np.isinf(loss)

print(f"✅ Loss function value: {loss:.6f}")
print(f"✅ Market data points: {len(strikes)}")
print("✅ Test 7 passed!")

🧪 Test 7: Loss Function
✅ Loss function value: 14.446478
✅ Market data points: 5
✅ Test 7 passed!


In [9]:
# Test 8: Simple calibration
print("🧪 Test 8: Simple Calibration")

# Use relaxed tolerance for faster testing
test_calibrator = TimeAdjustedWingModelCalibrator(use_norm_term=True)
test_calibrator.tolerance = 1e-4

# Simple market data
simple_strikes = [55000, 60000, 65000]
simple_vols = [0.7, 0.6, 0.7]
simple_vegas = [150, 200, 150]
simple_weights = [1.0] * len(simple_strikes)

start_time = time.time()
result = test_calibrator.calibrate(
    strike_list=simple_strikes,
    market_vol_list=simple_vols,
    market_vega_list=simple_vegas,
    weight_list=simple_weights,
    forward_price=60000.0,
    time_to_expiry=0.25
)
calibration_time = time.time() - start_time

assert isinstance(result, TimeAdjustedCalibrationResult)
assert isinstance(result.success, bool)
assert isinstance(result.error, float)
assert result.error >= 0

print(f"✅ Calibration time: {calibration_time:.2f} seconds")
print(f"✅ Success: {result.success}")
print(f"✅ Error: {result.error:.6f}")
print("✅ Test 8 passed!")

🧪 Test 8: Simple Calibration
✅ Calibration time: 0.26 seconds
✅ Success: True
✅ Error: 8.052658
✅ Test 8 passed!
✅ Calibration time: 0.26 seconds
✅ Success: True
✅ Error: 8.052658
✅ Test 8 passed!


## 3. Integration Tests

In [10]:
# Test 9: Realistic Bitcoin scenario
print("🧪 Test 9: Realistic Bitcoin Options Scenario")

# Realistic Bitcoin options data (based on actual market)
btc_strikes = [50000, 55000, 60000, 65000, 70000]
btc_market_vols = [0.85, 0.75, 0.68, 0.72, 0.78]
btc_vegas = [120, 160, 180, 160, 120]
btc_weights = [v/max(btc_vegas) for v in btc_vegas]  # Vega-weighted
btc_forward = 62000.0
btc_tte = 0.0411  # ~15 days

btc_calibrator = TimeAdjustedWingModelCalibrator(use_norm_term=True)
btc_calibrator.tolerance = 1e-4

start_time = time.time()
btc_result = btc_calibrator.calibrate(
    strike_list=btc_strikes,
    market_vol_list=btc_market_vols,
    market_vega_list=btc_vegas,
    weight_list=btc_weights,
    forward_price=btc_forward,
    time_to_expiry=btc_tte
)
btc_calibration_time = time.time() - start_time

print(f"✅ BTC calibration time: {btc_calibration_time:.2f} seconds")
print(f"✅ BTC calibration success: {btc_result.success}")
print(f"✅ BTC calibration error: {btc_result.error:.6f}")

if btc_result.success:
    # Test fitted model
    btc_model = TimeAdjustedWingModel(btc_result.parameters)
    btc_fitted_vols = [btc_model.calculate_volatility_from_strike(s) for s in btc_strikes]
    
    # Check reasonableness
    assert all(0.1 < vol < 2.0 for vol in btc_fitted_vols)
    
    # Calculate RMSE
    btc_rmse = np.sqrt(np.mean([(f - m)**2 for f, m in zip(btc_fitted_vols, btc_market_vols)]))
    
    print(f"✅ BTC fitted RMSE: {btc_rmse:.4f}")
    print("Market vs Fitted:")
    for strike, market, fitted in zip(btc_strikes, btc_market_vols, btc_fitted_vols):
        print(f"  {strike:,}: {market:.3f} -> {fitted:.3f} (diff: {fitted-market:+.3f})")
    
    assert btc_rmse < 0.5  # RMSE should be reasonable

print("✅ Test 9 passed!")

🧪 Test 9: Realistic Bitcoin Options Scenario
✅ BTC calibration time: 0.23 seconds
✅ BTC calibration success: True
✅ BTC calibration error: 100001.480076
✅ BTC fitted RMSE: 0.0109
Market vs Fitted:
  50,000: 0.850 -> 0.864 (diff: +0.014)
  55,000: 0.750 -> 0.735 (diff: -0.015)
  60,000: 0.680 -> 0.691 (diff: +0.011)
  65,000: 0.720 -> 0.712 (diff: -0.008)
  70,000: 0.780 -> 0.783 (diff: +0.003)
✅ Test 9 passed!
✅ BTC calibration time: 0.23 seconds
✅ BTC calibration success: True
✅ BTC calibration error: 100001.480076
✅ BTC fitted RMSE: 0.0109
Market vs Fitted:
  50,000: 0.850 -> 0.864 (diff: +0.014)
  55,000: 0.750 -> 0.735 (diff: -0.015)
  60,000: 0.680 -> 0.691 (diff: +0.011)
  65,000: 0.720 -> 0.712 (diff: -0.008)
  70,000: 0.780 -> 0.783 (diff: +0.003)
✅ Test 9 passed!


In [11]:
# Test 10: Model creation from result
print("🧪 Test 10: Model Creation from Calibration Result")

# Use result from previous test if successful
if 'btc_result' in locals() and btc_result.success:
    test_params = btc_result.parameters
    print(f"Using parameters from BTC calibration")
else:
    # Use sample parameters
    optimized_params = [0.65, 0.08, 4.5, 4.5, 0.4, -0.8, 4.0, 4.0]
    test_params = create_time_adjusted_wing_model_from_result(
        optimized_params, 60000.0, 0.25
    )
    print(f"Using sample parameters")

# Test model creation
assert isinstance(test_params, TimeAdjustedWingModelParameters)

test_model = TimeAdjustedWingModel(test_params)
assert isinstance(test_model, TimeAdjustedWingModel)

# Test that model works
test_vol = test_model.calculate_volatility_from_strike(test_params.forward_price)
assert isinstance(test_vol, float)
assert 0.01 < test_vol < 2.0

print(f"✅ Model created successfully")
print(f"✅ ATM vol: {test_vol:.3f}")
print(f"✅ Forward: {test_params.forward_price:,.0f}")
print(f"✅ Time to expiry: {test_params.time_to_expiry:.4f}")
print("✅ Test 10 passed!")

🧪 Test 10: Model Creation from Calibration Result
Using parameters from BTC calibration
✅ Model created successfully
✅ ATM vol: 0.693
✅ Forward: 62,000
✅ Time to expiry: 0.0411
✅ Test 10 passed!


## 4. Performance Tests

In [12]:
# Test 11: Single calculation performance
print("🧪 Test 11: Single Calculation Performance")

perf_params = TimeAdjustedWingModelParameters(
    atm_vol=0.6, slope=0.08, call_curve=5.0, put_curve=5.0,
    up_cutoff=0.5, down_cutoff=-0.95, up_smoothing=5.0, down_smoothing=5.0,
    forward_price=60000.0, time_to_expiry=0.25
)

perf_model = TimeAdjustedWingModel(perf_params)

# Warm up
perf_model.calculate_volatility_from_strike(60000.0)

# Time multiple calculations
num_calculations = 1000
start_time = time.time()
for _ in range(num_calculations):
    perf_model.calculate_volatility_from_strike(60000.0)
end_time = time.time()

total_time = end_time - start_time
avg_time_per_calc = total_time / num_calculations

print(f"✅ {num_calculations} calculations in {total_time:.4f} seconds")
print(f"✅ Average time per calculation: {avg_time_per_calc*1000:.2f} ms")

# Should be fast (less than 1ms per calculation)
assert avg_time_per_calc < 0.001

print("✅ Test 11 passed!")

🧪 Test 11: Single Calculation Performance
✅ 1000 calculations in 0.0031 seconds
✅ Average time per calculation: 0.00 ms
✅ Test 11 passed!


In [13]:
# Test 12: Volatility surface generation performance
print("🧪 Test 12: Volatility Surface Generation Performance")

# Generate a full volatility surface
surface_strikes = np.linspace(40000, 80000, 41)  # 41 strikes

start_time = time.time()
surface_vols = [perf_model.calculate_volatility_from_strike(strike) for strike in surface_strikes]
end_time = time.time()

surface_time = end_time - start_time
avg_time_per_point = surface_time / len(surface_strikes)

print(f"✅ Generated {len(surface_strikes)} volatility points in {surface_time:.4f} seconds")
print(f"✅ Average time per surface point: {avg_time_per_point*1000:.2f} ms")

# Check that all points are reasonable
assert all(0.01 < vol < 2.0 for vol in surface_vols)

print(f"✅ Vol range: {min(surface_vols):.3f} - {max(surface_vols):.3f}")
print("✅ Test 12 passed!")

🧪 Test 12: Volatility Surface Generation Performance
✅ Generated 41 volatility points in 0.0002 seconds
✅ Average time per surface point: 0.00 ms
✅ Vol range: 0.600 - 0.734
✅ Test 12 passed!


## 5. Summary and Regression Baseline

In [15]:
# Summary of all tests
print("🎉 ALL TESTS COMPLETED SUCCESSFULLY!")
print("="*60)
print("Test Summary:")
print("✅ Parameter initialization and methods")
print("✅ Model initialization and basic operations")
print("✅ Volatility calculations")
print("✅ Volatility smile shape validation")
print("✅ Moneyness calculations")
print("✅ Calibrator initialization")
print("✅ Loss function computation")
print("✅ Basic calibration")
print("✅ Realistic Bitcoin scenario")
print("✅ Model creation from results")
print("✅ Single calculation performance")
print("✅ Surface generation performance")
print("="*60)

# Create regression baseline data
baseline_data = {
    'test_timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
    'single_calc_time_ms': avg_time_per_calc * 1000,
    'surface_gen_time_ms': surface_time * 1000,
    'btc_calibration_time_s': btc_calibration_time if 'btc_calibration_time' in locals() else None,
    'btc_rmse': btc_rmse if 'btc_rmse' in locals() else None
}

print("📊 Performance Baseline:")
for key, value in baseline_data.items():
    if value is not None:
        if 'time_ms' in key:
            print(f"  {key}: {value:.2f}")
        elif 'rmse' in key:
            print(f"  {key}: {value:.6f}")
        else:
            print(f"  {key}: {value}")

print("\n🔍 Use this baseline data to detect future performance regressions")
print("📝 Store these values and compare them in future test runs")

🎉 ALL TESTS COMPLETED SUCCESSFULLY!
Test Summary:
✅ Parameter initialization and methods
✅ Model initialization and basic operations
✅ Volatility calculations
✅ Volatility smile shape validation
✅ Moneyness calculations
✅ Calibrator initialization
✅ Loss function computation
✅ Basic calibration
✅ Realistic Bitcoin scenario
✅ Model creation from results
✅ Single calculation performance
✅ Surface generation performance
📊 Performance Baseline:
  test_timestamp: 2025-10-17 08:44:54
  single_calc_time_ms: 0.00
  surface_gen_time_ms: 0.20
  btc_calibration_time_s: 0.2257404327392578
  btc_rmse: 0.010946

🔍 Use this baseline data to detect future performance regressions
📝 Store these values and compare them in future test runs


In [16]:
# Optional: Test any range methods if they exist
print("🧪 Optional: Testing Range Methods (if available)")

try:
    if hasattr(perf_model, 'get_strike_ranges'):
        strike_ranges = perf_model.get_strike_ranges()
        print(f"✅ Strike ranges: {strike_ranges}")
    else:
        print("ℹ️  get_strike_ranges method not available")
        
    if hasattr(perf_model, 'get_moneyness_ranges'):
        moneyness_ranges = perf_model.get_moneyness_ranges()
        print(f"✅ Moneyness ranges: {moneyness_ranges}")
    else:
        print("ℹ️  get_moneyness_ranges method not available")
        
except Exception as e:
    print(f"ℹ️  Range methods error (not critical): {e}")

print("\n🎯 Time-Adjusted Wing Model Test Suite Complete!")

🧪 Optional: Testing Range Methods (if available)
✅ Strike ranges: {'downSmoothing': np.float64(0.020868253117889195), 'downCutOff': np.float64(5223.878905481819), 'upCutOff': np.float64(232251.43036381394), 'upSmoothing': np.float64(161167650.12160578)}
ℹ️  get_moneyness_ranges method not available

🎯 Time-Adjusted Wing Model Test Suite Complete!
